<a href="https://colab.research.google.com/github/2395959141/evosax/blob/test1/%E2%80%9CEvotf_ES_brax_control_test_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC2024_9_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [3]:
!pip install evojax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.7/269.7 kB 5.0 MB/s eta 0:00:00


In [4]:
!pip install brax


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 998.9/998.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.6/696.6 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 18.4 MB/s eta 0:00:00
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94507 sha256=53d5710cb4ffe1a95aaefff1dfe54017b096b254ed81d3380f3134f72850dd1d

In [5]:
!pip install evosax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 kB 2.2 MB/s eta 0:00:00


EvoTF_ES With MLP Controller

In [2]:
import jax
import time
import numpy as np
from evojax.obs_norm import ObsNormalizer
from evojax.sim_mgr import SimManager
from evojax.task.brax_task import BraxTask
from evojax.policy import MLPPolicy

from evosax import Strategies
from evosax.utils.evojax_wrapper import Evosax2JAX_Wrapper

In [4]:
def get_brax_task(
    env_name = "humanoid",
    hidden_dims = [8, 8, 8, 8],
):
    train_task = BraxTask(env_name, test=False)
    test_task = BraxTask(env_name, test=True)
    policy = MLPPolicy(
        input_dim=train_task.obs_shape[0],
        output_dim=train_task.act_shape[0],
        hidden_dims=hidden_dims,
    )
    return train_task, test_task, policy

In [5]:
print(jax.devices())

[cuda(id=0)]


In [6]:
train_task, test_task, policy = get_brax_task("humanoid")
solver = Evosax2JAX_Wrapper(
    Strategies["EvoTF_ES"],
    param_size=policy.num_params,
    pop_size=5,
    es_config={"maximize": True,
               "centered_rank": True},
    #            "lrate_init": 0.01,
    #            "lrate_decay": 0.999,
    #            "lrate_limit": 0.001},
    # es_params={"sigma_init": 0.05,
    # "sigma_decay": 0.999,
    # "sigma_limit": 0.01},
    seed=0,
)
obs_normalizer = ObsNormalizer(
    obs_shape=train_task.obs_shape, dummy=not True
)
sim_mgr = SimManager(
    policy_net=policy,
    train_vec_task=train_task,
    valid_vec_task=test_task,
    seed=0,
    obs_normalizer=obs_normalizer,
    pop_size=5,
    use_for_loop=True,  #使用 Python 的 for 循环来进行模拟，那么可以将 use_for_loop 设置为 True。
    #如果你希望利用 JAX 的自动微分和并行计算功能，那么可以将 use_for_loop 设置为 False。
    n_repeats=8,
    test_n_repeats=1,
    n_evaluations=32,
)

print(f"START EVOLVING {policy.num_params} PARAMS.")
# Run ES Loop.
for gen_counter in range(10):
    start_time = time.time()  # 记录开始时间

    params = solver.ask()
    scores, _ = sim_mgr.eval_params(params=params, test=False)
    solver.tell(fitness=scores)
    if gen_counter == 0 or (gen_counter + 1) % 2 == 0:
        test_scores, _ = sim_mgr.eval_params(
            params=solver.best_params, test=True
        )
        print(
            {
                "num_gens": gen_counter + 1,
            },
            {
                "train_perf": float(np.nanmean(scores)),
                "test_perf": float(np.nanmean(test_scores)),
            },
        )
        end_time = time.time()  # 记录结束时间
        elapsed_time = end_time - start_time  # 计算时间差
        print(f"Time taken for iteration {gen_counter + 1}: {elapsed_time} seconds")

INFO:MLPPolicy:MLPPolicy.num_params = 2329


Loaded pretrained EvoTF model from ckpt: 2024_03_SNES_small.pkl


INFO:SimManager:use_for_loop=True


START EVOLVING 2329 PARAMS.
{'num_gens': 1} {'train_perf': 82.9275894165039, 'test_perf': 92.17207336425781}
Time taken for iteration 1: 81.99760818481445 seconds
{'num_gens': 2} {'train_perf': 88.8930435180664, 'test_perf': 119.46363067626953}
Time taken for iteration 2: 0.9399583339691162 seconds
{'num_gens': 4} {'train_perf': 103.92070007324219, 'test_perf': 90.72538757324219}
Time taken for iteration 4: 1.1833908557891846 seconds
{'num_gens': 6} {'train_perf': 117.50395202636719, 'test_perf': 150.63670349121094}
Time taken for iteration 6: 1.245741844177246 seconds
{'num_gens': 8} {'train_perf': 92.0579605102539, 'test_perf': 95.21200561523438}
Time taken for iteration 8: 0.7630395889282227 seconds
{'num_gens': 10} {'train_perf': 120.59161376953125, 'test_perf': 152.20315551757812}
Time taken for iteration 10: 1.3475048542022705 seconds


In [7]:
test_scores, _ = sim_mgr.eval_params(
            params=solver.best_params, test=True
        )
print(
    {
        "num_gens": gen_counter + 1,
    },
    {
        "train_perf": float(np.nanmean(scores)),
        "test_perf": float(np.nanmean(test_scores)),
    },
)

{'num_gens': 10} {'train_perf': 120.59161376953125, 'test_perf': 153.47531127929688}


In [9]:
from IPython.display import HTML
from brax import envs
from brax.io import html
import jax
from brax.io import model
from brax.io import json

env = envs.create(env_name="ant")
task_reset_fn = jax.jit(env.reset)
policy_reset_fn = jax.jit(policy.reset)
step_fn = jax.jit(env.step)
act_fn = jax.jit(policy.get_actions)
obs_norm_fn = jax.jit(obs_normalizer.normalize_obs)

best_params = solver.best_params
obs_params = sim_mgr.obs_params

total_reward = 0
rollout = []
rng = jax.random.PRNGKey(seed=42)
task_state = task_reset_fn(rng=rng)
policy_state = policy_reset_fn(task_state)
while not task_state.done:
    rollout.append(task_state)
    task_state = task_state.replace(
    obs=obs_norm_fn(task_state.obs[None, :], obs_params).reshape(1, 27))
    act, policy_state = act_fn(task_state, best_params[None, :], policy_state)
    task_state = task_state.replace(
    obs=obs_norm_fn(task_state.obs[None, :], obs_params).reshape(27,))
    task_state = step_fn(task_state, act[0])
    total_reward = total_reward + task_state.reward

print("Cumulative reward:", total_reward)
HTML(html.render(env.sys, [s.pipeline_state for s in rollout]))
#HTML(html.render(env.sys, [rollout.state.pipeline_state]))

ValueError: Incompatible shapes for broadcasting: shapes=[(1, 27), (244,)]